<a href="https://colab.research.google.com/github/NoamMichael/Comparing-Confidence-in-LLMs/blob/main/DatasetFormatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Notebook to format datasets:

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
## Output Format:
## | Question ID | Question | Option A | Option B | ... | Correct Answer Letter |
## |     (Int)       |     (Str)     |  (Str)   |  (Str)   |     |       (Char)          |

In [8]:
%pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [51]:
import pandas as pd
import numpy as np
import huggingface_hub
import re
from datasets import load_dataset

## Datasets Using:
## LSAT-AR
## SciQ
## BoolQ
## MMLU
## WGD
## HaluEval

dataset_ids = {
    'LSAT-AR': {
        'Key':'hails/agieval-lsat-ar',
        'Link': 'https://huggingface.co/datasets/hails/agieval-lsat-ar'
    },
    'SciQ': { ## TODO
        'Key':'allenai/sciq',
        'Link': 'https://huggingface.co/datasets/allenai/sciq'
    },
    'BoolQ': { ## TODO
        'Key':'google/boolq',
        'Link': 'https://huggingface.co/datasets/google/boolq'
    },
    'MMLU': { ## TODO Maybe not worth our time to do?
        'Key':'hails/agieval-sciq',
        'Link': 'https://huggingface.co/datasets/hails/agieval-sciq'
    },
    #'WGD': 'wgd', #Not on huggingface yet
    'HaluEval': { ## TODO
        'Key':'hails/agieval-sciq',
        'Link': 'https://huggingface.co/datasets/hails/agieval-sciq'
    },
}



In [49]:
## LSAT:
lsat_code = dataset_ids['LSAT-AR']['Key']
lsat = load_dataset(lsat_code)
lsat_orig_df = pd.DataFrame(lsat['test'])
lsat_df = lsat_orig_df.copy()

# Question ID
lsat_df['Question ID'] = lsat_df.index

# Question
lsat_df['Question'] = lsat_df['query']
lsat_df = lsat_df.drop(columns=['query'])

# Option collumns

for i in range(len(lsat_df['choices'][0])): # Assuming all rows have the same number of choices
    lsat_df[f'Option {chr(65+i)}'] = lsat_df['choices'].apply(lambda x: re.sub(r'\([A-Z]\)', '', x[i], count=1))


lsat_df = lsat_df.drop(columns=['choices'])


# Correct Answer Letter
lsat_df['Correct Answer Letter'] = lsat_df['gold'].apply(lambda x: x[0]).apply(lambda x: chr(x + 65))
lsat_df = lsat_df.drop(columns=['gold'])



display(lsat_df.tail())
name = 'lsat_ar_test_formatted.csv'
lsat_df.to_csv(name, index=False)

,Question ID,Question,Option A,Option B,Option C,Option D,Option E,Correct Answer Letter
225,225,"Three real estate companies—RealProp, Southco,...",RealProp: the Flores Tower and the Garza Tower...,"RealProp: the Garza Tower, the King Building, ...",RealProp: the Garza Tower and the Lynch Buildi...,"RealProp: the Garza Tower, the Meyer Building,...","RealProp: the Garza Tower, the Yates House, an...",C
226,226,"Three real estate companies—RealProp, Southco,...",The buildings owned by RealProp are the Flores...,The buildings owned by Southco are the Flores ...,The buildings owned by Southco are the Garza T...,The buildings owned by Trustcorp are the Flore...,The buildings owned by Trustcorp are the Garza...,A
227,227,"Three real estate companies—RealProp, Southco,...",Trustcorp owns a class 1 building.,Trustcorp owns the Meyer Building.,Southco owns a class 2 Building.,Southco owns both of the class 3 buildings.,Southco owns the Flores Tower.,A
228,228,"Three real estate companies—RealProp, Southco,...",RealProp owns a class 1 building.,Southco owns only class 2 buildings.,Southco has made at least one trade with Trust...,Trustcorp owns the Garza Tower.,Trustcorp owns the Zimmer House.,E
229,229,"Three real estate companies—RealProp, Southco,...",The buildings owned by RealProp are the Lynch ...,The buildings owned by Southco are the Garza T...,The buildings owned by Southco are the King Bu...,The buildings owned by Trustcorp are the Flore...,The buildings owned by Trustcorp are the Garza...,D


In [78]:
np.random.seed(42)

## SciQ
sciq_code = dataset_ids['SciQ']['Key']
sciq = load_dataset(sciq_code)
sciq_orig_df = pd.DataFrame(sciq['test'])
sciq_df = sciq_orig_df.copy()

## Drop Support
sciq_df = sciq_df.drop(columns=['support'])

## Correct Answer Letter

numbers = []
for i in range(len(sciq_df)):
    numbers.append(np.random.randint(0, 4))

sciq_df['Correct Answer Letter'] = numbers

## Options column:
def create_options(row):

    correct_answer_index = row['Correct Answer Letter']
    correct_answer = row['correct_answer']
    dist1 = row['distractor1']
    dist2 = row['distractor2']
    distr3 = row['distractor3']

    # Create a list of just the distractors initially
    options = [dist1, dist2, distr3]

    # Insert the correct answer
    options.insert(correct_answer_index, correct_answer)
    return options

sciq_df['Options'] = sciq_df.apply(create_options, axis=1)

for i in range(len(sciq_df['Options'][0])):
    sciq_df[f'Option {chr(65+i)}'] = sciq_df['Options'].apply(lambda x: re.sub(r'\([A-Z]\)', '', x[i], count=1))





sciq_df['Correct Answer Letter'] = sciq_df['Correct Answer Letter'].apply(lambda x: chr(x + 65))
sciq_df['Question ID'] = sciq_df.index
sciq_df['Question'] = sciq_df['question']
sciq_df = sciq_df.drop(columns=['question'])

## Drop old columns
sciq_df = sciq_df.drop(columns=['distractor1', 'distractor2', 'distractor3', 'correct_answer', 'Options'])

## Re order columns for neatness

desired_order = [
    'Question ID',
    'Question',
    'Option A',
    'Option B',
    'Option C',
    'Option D',
    'Correct Answer Letter'
]

sciq_df = sciq_df[desired_order]

name = 'sciq_test_formatted.csv'
sciq_df.to_csv(name, index=False)


sciq_df#.head()

,Question ID,Question,Option A,Option B,Option C,Option D,Correct Answer Letter
0,0,Compounds that are capable of accepting electr...,antioxidants,Oxygen,oxidants,residues,C
1,1,What term in biotechnology means a genetically...,adult,male,phenotype,clone,D
2,2,Vertebrata are characterized by the presence o...,backbone,Bones,Muscles,Thumbs,A
3,3,What is the height above or below sea level ca...,depth,latitude,elevation,variation,C
4,4,"Ice cores, varves and what else indicate the e...",mountain ranges,fossils,tree rings,magma,C
...,...,...,...,...,...,...,...
995,995,"In the case of the moose, predation is an addi...",population,speed,color,weight,A
996,996,Where do short period comets come from?,kuiper belt,milky way,vesta belt,photon belt,A
997,997,Only after implantation can an embryo develop ...,mammal,humans,living thing,fetus,D
998,998,What are atoms with unstable nuclei are consid...,ions,unstable,destructive,radioactive,D


In [85]:
## BoolQ

# Load the BoolQ dataset
boolq_code = dataset_ids['BoolQ']['Key']
boolq = load_dataset(boolq_code)
boolq_orig_df = pd.DataFrame(boolq['validation'])
boolq_df = boolq_orig_df.copy()

# Question ID
boolq_df['Question ID'] = boolq_df.index


boolq_df.rename(columns={'question': 'Question', 'answer': 'Correct Answer'}, inplace=True)
boolq_df.drop(columns=['passage'], inplace=True)

boolq_df.head()

## Save as CSV

boolq_df.to_csv('boolq_valid_formatted.csv', index=False)

